# The Price to Earnings Ratio (P/E) model

This Notebook will be used for calculating the Price to Earnings (P/E) model as described in the Medium Post [Sure you know the stock price. But do you know its value?](https://medium.com/@sankha.mukherjee_007/sure-you-know-the-stock-price-but-do-you-know-its-value-65ab44dadd04) in Section 4.4. 

This implementation will be as close as possible to the original article so that one is able to easily follow the original article and this one.

In [1]:
import financeMacroFactors as fM
import pandas as pd
import numpy as np
from scipy import interpolate
from datetime import datetime as dt
from datetime import timedelta as tDel
import matplotlib.pyplot as plt

In [2]:
fundamentals = fM.companies.getTickerFundamentalDataMW('MSFT')

## Download & Calculate Fundamental Data

Before we do the calculations, we should download and calculate the fundamental data that we need. Here as well, we shall follow the article as closely as possible.

### Download the Earnings Per Share (EPS)

The EPS $\mathbf{e}$ will be placed in the Python variable `e`:

In [3]:
earningsPerShare = fM.companies.extractYearlyData(fundamentals['IncomeStatement'], 'EPS (Diluted)')
y, e = zip(*earningsPerShare)
e = np.array(e)
print(f'--------[Revenue]------------')
print(f'Years: {[m.year for m in y]}')
print(f'EPS during that time: e = {e}')

--------[Revenue]------------
Years: [2016, 2017, 2018, 2019, 2020]
EPS during that time: e = [2.1  3.25 2.13 5.06 5.76]


### Download the stock price 

The stock proce $\mathbf{s}$ is placed in the python variable `p`. Note that the stock proce has two properties that one needs to take care of:

1. Monthly-averaged stock price is averaged on the first of the month, while the fundamental data is available at the end of the month. For this reason, the share price will have to be typically shifted by a day to match the dates of the fundamental data. The averaged stock price data is for the first of the *next month*.
2. The stock data is arranged such that the dates are descending, while the fundamental data is arranged such that the dates are ascending. To fix this problem. the stock data is reversed.


In [4]:
stockPrice = fM.companies.yahooData.getStockDataYahoo('msft', y[0]-tDel(90), dt.now(), '1mo')

In [5]:
stockPrice = stockPrice[1:]
p = np.array([m[-2] for m in stockPrice if ((m[0]-tDel(1)) in y)])
p = p[::-1]
print(f'Stock Price over the same period = {p}')

Stock Price over the same period = [ 52.6   69.15 102.87 134.29 204.52]


## Calculate the P/E Ratio

This is the stock price ($\mathbf{p}$) divided by the revenue/stock ($\mathbf e$). Note that all operations are element-wise operations. From this point on, there are only two more operations to perform:

### Calculate the P/E ratio over the last few years


In [6]:
pe = p/e
pe

array([25.04761905, 21.27692308, 48.29577465, 26.53952569, 35.50694444])

### Use the average P/E ratio to claculate the P/E valuation

In [7]:
pe.mean()*(e[-1])

180.48013851867665